<a href="https://colab.research.google.com/github/ltpdo/GradeData-MachineLearing/blob/formulas/%E7%A7%91%E7%9B%AE%E3%81%94%E3%81%A8%E3%81%AE%E6%88%90%E7%B8%BE%E7%AE%97%E5%87%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ライブラリのインポート

In [5]:
!pip install gradio
!pip install gspread

In [99]:
# ライブラリのインポート
import gradio as gr
import gspread
import pandas as pd
import numpy as np

In [9]:
# 認証のためのコード
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [11]:
# Google Driveとの接続
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/卒論/成績予測する科目/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# データの前処理

In [94]:
# グーグルスプレッドシートをurlで開く
url = "https://docs.google.com/spreadsheets/d/1u0DKe-96bHj64VYDR3JPBT2m610ASQhgCzyZxDc1c0w/edit#gid=64615062"
workbook = gc.open_by_url(url)

# シートの特定を行う
sheet = workbook.worksheet("科目ごとの回帰係数(削除版)")

In [95]:
# スプレッドシートのデータをPandasのDataframeに変換する。
rows = sheet.get_all_values()
df = pd.DataFrame.from_records(rows)
df

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,,キャリア形成基礎論,コンピュータネットワーク,データ構造とアルゴリズムⅠ,データ構造とアルゴリズムⅠ演習,ハードウェア実験Ⅰ,ハードウェア設計Ⅰ,ハードウェア設計Ⅱ,プログラミング入門,プログラミング基礎Ⅰ,...,数理的教養Ⅰ,線形代数Ⅰ,線形代数Ⅱ,計算機構成論Ⅰ,計算機構成論Ⅱ,離散数学Ⅰ,離散数学Ⅱ,離散数学Ⅲ,ＡＩプログラミング演習,情報理論・確率論
1,切片,-0.467,-0.098,-0.116,-0.195,-0.527,0.078,0.206,-0.013,0.013,...,0.02,0.021,-0.089,-0.042,-0.107,-0.049,0.175,0.175,0.619,0.756
2,回帰係数1,微分積分Ⅰ: 0.542419,基礎物理 0.311357,計算機構成論Ⅱ 0.249704,組込みソフトウェア演習 0.422306,モバイルプログラミング演習 0.718769,計算機構成論Ⅰ 0.192538,計算機構成論Ⅱ 0.230415,モバイルプログラミング演習: 0.256244,線形代数Ⅰ 0.851749,...,基礎ゼミナール 0.776533,データ構造とアルゴリズムⅠ演習 0.086488,線形代数Ⅰ 2.083490,線形代数Ⅰ 1.175108,計算機構成論Ⅰ 0.404537,微分積分Ⅰ 0.260801,線形代数Ⅰ 2.157348,線形代数Ⅰ 2.157348,スポーツ科学演習 1.351449,情報リテラシー 1.241754
3,回帰係数2,プログラミング基礎Ⅱ: 0.529624,情報理論・確率論 0.303190,ＡＩプログラミング演習 0.221179,離散数学Ⅰ 0.392583,基礎物理 0.613128,離散数学Ⅰ 0.109622,情報理論・確率論 0.187705,基礎数学: 0.226994,離散数学Ⅰ 0.793431,...,プログラミング基礎Ⅰ 0.427632,情報科学基礎演習 0.081493,基礎数学 1.253758,プログラミング入門 1.153717,離散数学Ⅰ 0.232634,モバイルプログラミング演習 0.206175,離散数学Ⅰ 1.071187,離散数学Ⅰ 1.071187,計算機構成論Ⅰ 1.030660,スポーツ科学演習 1.119312
4,回帰係数3,基礎物理: 0.498471,計算機構成論Ⅱ 0.236033,日本語表現法 0.195909,プログラミング基礎Ⅱ 0.360221,ＡＩプログラミング演習 0.410830,微分積分Ⅰ 0.108994,ハードウェア設計Ⅰ 0.175978,プログラミング基礎Ⅰ: 0.149907,プログラミング入門 0.644141,...,線形代数Ⅰ 0.390762,離散数学Ⅱ 0.066196,基礎ゼミナール 0.901302,基礎数学 0.681188,微分積分Ⅰ 0.228420,データ構造とアルゴリズムⅠ演習 0.205291,情報リテラシー 0.715022,情報リテラシー 0.715022,モバイルプログラミング演習 1.029847,プログラミング入門 0.376361
5,回帰係数4,微分積分Ⅱ: 0.446643,ハードウェア設計Ⅰ 0.157423,基礎物理 0.164167,日本語表現法 0.338522,組込みソフトウェア演習 0.376113,キャリア形成基礎論 0.096876,スポーツ科学演習 0.154934,データ構造とアルゴリズムⅠ演習: 0.138364,スポーツ科学演習 0.483545,...,離散数学Ⅰ 0.370972,離散数学Ⅲ 0.060726,スポーツ科学演習 0.871003,離散数学Ⅰ 0.301170,キャリア形成基礎論 0.218947,基礎数学 0.138580,キャリア形成基礎論 0.449792,キャリア形成基礎論 0.449792,基礎物理 0.969927,基礎物理 0.335848
6,回帰係数5,ハードウェア実験Ⅰ: 0.411114,キャリア形成基礎論 0.119175,基礎数学 0.160515,基礎数学 0.337622,基礎数学 0.375078,数理的教養Ⅰ 0.085246,プログラミング基礎Ⅱ 0.142836,情報理論・確率論: 0.132085,キャリア形成基礎論 0.454249,...,モバイルプログラミング演習 0.342682,基礎ゼミナール 0.057468,プログラミング入門 0.628600,スポーツ科学演習 0.041714,基礎ゼミナール 0.184652,線形代数Ⅰ 0.136135,スポーツ科学演習 0.398879,スポーツ科学演習 0.398879,情報リテラシー 0.794864,線形代数Ⅱ 0.305905
7,回帰係数6,スポーツ科学演習: 0.398480,微分積分Ⅱ 0.117778,計算機構成論Ⅰ 0.157134,ＡＩプログラミング演習 0.320160,キャリア形成基礎論 0.349367,線形代数Ⅰ 0.075823,プログラミング入門 0.108172,ハードウェア設計Ⅱ: 0.118079,基礎数学 0.239499,...,スポーツ科学演習 0.322957,プログラミング基礎Ⅰ 0.051494,キャリア形成基礎論 0.174305,情報リテラシー 0.037965,モバイルプログラミング演習 0.154322,微分積分Ⅱ 0.132865,プログラミング入門 0.392469,プログラミング入門 0.392469,プログラミング入門 0.684514,基礎数学 0.266411
8,回帰係数7,線形代数Ⅰ: 0.354710,線形代数Ⅱ 0.113565,離散数学Ⅱ 0.122502,プログラミング入門 0.280516,ハードウェア設計Ⅰ 0.316182,情報科学基礎演習 0.071784,モバイルプログラミング演習 0.105155,プログラミング基礎Ⅱ: 0.107258,情報リテラシー 0.087042,...,情報科学基礎演習 0.312719,情報リテラシー 0.051418,離散数学Ⅰ 0.161565,基礎ゼミナール 0.035059,プログラミング入門 0.127765,ＡＩプログラミング演習 0.126674,基礎ゼミナール 0.092053,基礎ゼミナール 0.092053,基礎ゼミナール 0.638172,キャリア形成基礎論 0.265963
9,回帰係数8,線形代数Ⅱ: 0.310491,統計学 0.103790,プログラミング入門 0.100162,ハードウェア設計Ⅰ 0.262953,プログラミング基礎Ⅱ 0.266639,基礎数学 0.071515,微分積分Ⅰ 0.100465,基礎ゼミナール: 0.105258,数理的教養Ⅰ 0.041223,...,線形代数Ⅱ 0.310800,計算機構成論Ⅰ 0.047000,情報リテラシー 0.155880,キャリア形成基礎論 0.006656,基礎数学 0.100932,ハードウェア設計Ⅰ 0.107216,基礎数学 0.046097,基礎数学 0.046097,微分積分Ⅰ 0.393172,プログラミング基礎Ⅰ 0.187989


In [149]:
# データフレームを変換
grade_sheet = df.T
grade_sheet.columns = grade_sheet.iloc[0]
grade_sheet = grade_sheet.iloc[1:]

# 空白をNaNに変換
grade_sheet.replace('', np.nan, inplace=True)

# 使用しない科目を削除
grade_sheet = grade_sheet.dropna(subset = ["test_size", "test"])

# 列を削除
grade_sheet = grade_sheet.dropna(axis=1, how='all')
grade_sheet = grade_sheet.drop(columns = ["test_size", "test"])

# indexに指定
grade_sheet.set_index(grade_sheet.iloc[:, 0], inplace = True)
grade_sheet.drop(columns = grade_sheet.columns[0], inplace = True)

# 欠損値を補完
grade_sheet =  grade_sheet.fillna(0)

# データの表示
grade_sheet

# データの保存
grade_sheet.to_csv('/content/drive/MyDrive/Colab Notebooks/卒論/成績予測する科目/方程式に使うデータフレーム.csv')

# データの数式化


In [144]:
# grade_sheet DataFrame を辞書型に変換します
data_dict = grade_sheet.to_dict(orient='index')

print(data_dict)

{'コンピュータネットワーク': {'切片': '-0.098', '回帰係数1': '基礎物理               0.311357', '回帰係数2': '情報理論・確率論           0.303190', '回帰係数3': '計算機構成論Ⅱ            0.236033', '回帰係数4': 'ハードウェア設計Ⅰ          0.157423', '回帰係数5': 'キャリア形成基礎論          0.119175', '回帰係数6': '微分積分Ⅱ              0.117778', '回帰係数7': '線形代数Ⅱ              0.113565', '回帰係数8': '統計学                0.103790', '回帰係数9': 'ＡＩプログラミング演習        0.095154', '回帰係数10': '日本語表現法             0.092831', '回帰係数11': 'プログラミング基礎Ⅱ         0.079273', '回帰係数12': '情報科学基礎演習           0.073856', '回帰係数13': '基礎数学               0.071869', '回帰係数14': '離散数学Ⅰ              0.065405', '回帰係数15': 'スポーツ科学演習           0.064598', '回帰係数16': '情報リテラシー            0.052042', '回帰係数17': 'プログラミング基礎Ⅰ         0.051477', '回帰係数18': 'モバイルプログラミング演習      0.036605', '回帰係数19': '数理的教養Ⅰ             0.034946', '回帰係数20': '組込みソフトウェア演習        0.034307', '回帰係数21': '計算機構成論Ⅰ            0.033903', '回帰係数22': 'プログラミング言語とコンパイラ    0.028462', '回帰係数23': '線形代数Ⅰ              0.028391', '回帰係数24': '離散数学Ⅲ              0.

In [148]:
# 辞書型を変換する関数
def convert_regression_coefficients(data, prefix='回帰係数', start_index=1):
    """
    回帰係数のデータを変換する関数。

    Args:
        data (dict): 変換するデータの辞書。
        prefix (str, optional): 回帰係数のキーのプレフィックス。デフォルトは'回帰係数'。
        start_index (int, optional): 回帰係数の数値の開始インデックス。デフォルトは1。

    Returns:
        dict: 変換されたデータの辞書。
    """
    # 変換されたデータを格納するための空の辞書を作成
    converted_data = {}

    # 元のデータの各要素に対して処理を行う
    for key, value in data.items():
        # 変換された回帰係数を格納するための空の辞書を作成
        converted_coefficients = {}

        # 切片を格納
        if '切片' in value:
            converted_coefficients['切片'] = float(value['切片'])

        # 各要素のキーに対して処理を行う
        for coefficient_key in value.keys():
            # 指定されたプレフィックスで始まるキーの場合
            if coefficient_key.startswith(prefix):
                # 対応する値を取得
                coefficient_value = value[coefficient_key]

                # 値が文字列であることを確認
                if isinstance(coefficient_value, str):
                    # 文字列を空白文字で分割し、科目名と係数に分ける
                    subject_key, coefficient_value = coefficient_value.split()

                    # 科目名をキーとして、係数を値として格納
                    converted_coefficients[subject_key] = float(coefficient_value)

        # 元のデータのキーに対応付け、変換された回帰係数を格納
        converted_data[key] = converted_coefficients

    # 変換されたデータを返す
    return converted_data



# 回帰係数のキーを科目名に変更
converted_data = convert_regression_coefficients(data_dict)

print(converted_data)

{'コンピュータネットワーク': {'切片': -0.098, '基礎物理': 0.311357, '情報理論・確率論': 0.30319, '計算機構成論Ⅱ': 0.236033, 'ハードウェア設計Ⅰ': 0.157423, 'キャリア形成基礎論': 0.119175, '微分積分Ⅱ': 0.117778, '線形代数Ⅱ': 0.113565, '統計学': 0.10379, 'ＡＩプログラミング演習': 0.095154, '日本語表現法': 0.092831, 'プログラミング基礎Ⅱ': 0.079273, '情報科学基礎演習': 0.073856, '基礎数学': 0.071869, '離散数学Ⅰ': 0.065405, 'スポーツ科学演習': 0.064598, '情報リテラシー': 0.052042, 'プログラミング基礎Ⅰ': 0.051477, 'モバイルプログラミング演習': 0.036605, '数理的教養Ⅰ': 0.034946, '組込みソフトウェア演習': 0.034307, '計算機構成論Ⅰ': 0.033903, 'プログラミング言語とコンパイラ': 0.028462, '線形代数Ⅰ': 0.028391, '離散数学Ⅲ': 0.027924, 'プログラミング入門': 0.027779, '離散数学Ⅱ': 0.025641, '微分積分Ⅰ': 0.017432, '基礎ゼミナール': 0.009304}, 'データ構造とアルゴリズムⅠ': {'切片': -0.116, '計算機構成論Ⅱ': 0.249704, 'ＡＩプログラミング演習': 0.221179, '日本語表現法': 0.195909, '基礎物理': 0.164167, '基礎数学': 0.160515, '計算機構成論Ⅰ': 0.157134, '離散数学Ⅱ': 0.122502, 'プログラミング入門': 0.100162, '統計学': 0.099507, 'ハードウェア設計Ⅰ': 0.096101, '離散数学Ⅰ': 0.093628, 'キャリア形成基礎論': 0.079364, '線形代数Ⅱ': 0.07119, 'スポーツ科学演習': 0.064112, '微分積分Ⅱ': 0.060764, 'プログラミング言語とコンパイラ': 0.047875, '離散

In [163]:
def generate_formula(data):
    """
    データから数式を生成する関数。

    Args:
        data (dict): 数式を生成するためのデータ。

    Returns:
        dict: 各データの数式を格納した辞書。
    """
    formulas = {}

    for category, values in data.items():
        formula = ""
        if '切片' in values:
            formula += f"{values['切片']}"

        for key, value in values.items():
            if key != '切片':
                if value != 0:
                    formula += f" + {value} * {key}"

        formula = formula.lstrip(' +')
        formulas[category] = formula

    return formulas

# 与えられたデータ
formula = generate_formula(converted_data)
print(formula)


{'コンピュータネットワーク': '-0.098 + 0.311357 * 基礎物理 + 0.30319 * 情報理論・確率論 + 0.236033 * 計算機構成論Ⅱ + 0.157423 * ハードウェア設計Ⅰ + 0.119175 * キャリア形成基礎論 + 0.117778 * 微分積分Ⅱ + 0.113565 * 線形代数Ⅱ + 0.10379 * 統計学 + 0.095154 * ＡＩプログラミング演習 + 0.092831 * 日本語表現法 + 0.079273 * プログラミング基礎Ⅱ + 0.073856 * 情報科学基礎演習 + 0.071869 * 基礎数学 + 0.065405 * 離散数学Ⅰ + 0.064598 * スポーツ科学演習 + 0.052042 * 情報リテラシー + 0.051477 * プログラミング基礎Ⅰ + 0.036605 * モバイルプログラミング演習 + 0.034946 * 数理的教養Ⅰ + 0.034307 * 組込みソフトウェア演習 + 0.033903 * 計算機構成論Ⅰ + 0.028462 * プログラミング言語とコンパイラ + 0.028391 * 線形代数Ⅰ + 0.027924 * 離散数学Ⅲ + 0.027779 * プログラミング入門 + 0.025641 * 離散数学Ⅱ + 0.017432 * 微分積分Ⅰ + 0.009304 * 基礎ゼミナール', 'データ構造とアルゴリズムⅠ': '-0.116 + 0.249704 * 計算機構成論Ⅱ + 0.221179 * ＡＩプログラミング演習 + 0.195909 * 日本語表現法 + 0.164167 * 基礎物理 + 0.160515 * 基礎数学 + 0.157134 * 計算機構成論Ⅰ + 0.122502 * 離散数学Ⅱ + 0.100162 * プログラミング入門 + 0.099507 * 統計学 + 0.096101 * ハードウェア設計Ⅰ + 0.093628 * 離散数学Ⅰ + 0.079364 * キャリア形成基礎論 + 0.07119 * 線形代数Ⅱ + 0.064112 * スポーツ科学演習 + 0.060764 * 微分積分Ⅱ + 0.047875 * プログラミング言語とコンパイラ + 0.042242 * 離散数

In [ ]:
# 入力
data_input = {
    "コンピューターネットワーク" : -1, "データ構造とアルゴリズムⅠ" : -1, "データ構造とアルゴリズムⅠ演習" : -1, "ハードウェア実験Ⅰ" : -1,
    "ハードウェア設計Ⅰ": -1, "ハードウェア設計Ⅱ" : -1, "プログラミング基礎Ⅰ" : -1, "プログラミング基礎Ⅱ" : -1,
    "モバイルプログラミング演習" : -1, "基礎物理" : -1, "微分積分Ⅰ" : -1, "微分積分Ⅱ" : -1, "数理的教養Ⅰ" : -1, "線形代数Ⅱ" : -1,
    "計算機構成論Ⅰ" : -1, "計算機構成論Ⅱ" : -1, "離散数学Ⅱ" : -1, "離散数学Ⅲ" : -1, "ＡＩプログラミング演習" : -1,
    "情報理論・確率論" : -1
    }

# 出力
data_output = {
    "コンピューターネットワーク" : -1, "データ構造とアルゴリズムⅠ" : -1, "データ構造とアルゴリズムⅠ演習" : -1, "ハードウェア実験Ⅰ" : -1,
    "ハードウェア設計Ⅰ": -1, "ハードウェア設計Ⅱ" : -1, "プログラミング基礎Ⅰ" : -1, "プログラミング基礎Ⅱ" : -1,
    "モバイルプログラミング演習" : -1, "基礎物理" : -1, "微分積分Ⅰ" : -1, "微分積分Ⅱ" : -1, "数理的教養Ⅰ" : -1, "線形代数Ⅱ" : -1,
    "計算機構成論Ⅰ" : -1, "計算機構成論Ⅱ" : -1, "離散数学Ⅱ" : -1, "離散数学Ⅲ" : -1, "ＡＩプログラミング演習" : -1,
    "情報理論・確率論" : -1
    }